In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config Completer.use_jedi=False

import json
import tqdm
import re
import string

import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import spacy

from sklearn.metrics.pairwise import cosine_similarity
import scipy


In [2]:
stop_words = set(stopwords.words('english'))

In [3]:
data = pd.read_csv(r"E:\DSP\DSP_22nd Nov\Machine Learning\FInal Class\Case Studies & FInal Projects\6\data job posts.csv")
data.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\nJOB TITL...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\nc...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\nI...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\nJOB...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\nn...,NaN,2004,1,False
3,Manoff Group\nJOB TITLE: BCC Specialist\nPOSI...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\nPe...,NaN,23 January 2004\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\nJOB TITLE: Software D...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\n- CV; \n-...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


In [4]:
data.shape

(19001, 24)

In [5]:
data = data.drop_duplicates(["jobpost","Title"])

In [6]:
data.shape

(18892, 24)

### Choosing Selective Columns

In [7]:
# Selecting only those columns which will be needed to evaluate the JOb similarity based on Cosine similarity scores
data_job_related = data[['Title','JobDescription','RequiredQual','JobRequirment']]

In [8]:
data_job_related.head()

,Title,JobDescription,RequiredQual,JobRequirment
0,Chief Financial Officer,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\nindividu...",- Supervises financial management and administ...
1,Full-time Community Connections Intern (paid i...,NaN,- Bachelor's Degree; Master's is preferred;\n-...,NaN
2,Country Coordinator,Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...
3,BCC Specialist,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...
4,Software Developer,NaN,- University degree; economical background is ...,- Rendering technical assistance to Database M...


In [9]:
data_job_related.shape

(18892, 4)

In [12]:
data_job_related.Title = data_job_related.Title.astype(str)
data_job_related.JobDescription = data_job_related.JobDescription.astype(str)
data_job_related.JobRequirment = data_job_related.JobRequirment.astype(str)
data_job_related.RequiredQual = data_job_related.RequiredQual.astype(str)

C:\Users\Akhtar\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [16]:
data_job_related["JobDescription_tokenize"] = data_job_related["JobDescription"].map(word_tokenize)

C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
data_job_related["JobRequirment_tokenize"] = data_job_related.JobRequirment.map(word_tokenize)

C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
data_job_related["RequiredQual_tokenize"] = data_job_related.RequiredQual.map(word_tokenize)

C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
# stopwords and punctuation removal for the 3 columns on the tokenized data 
data_job_related['JobDescription_tokenize'] = data_job_related['JobDescription_tokenize'].apply(lambda x: [item for item in x if item not in stop_words])
data_job_related['JobDescription_tokenize'] = data_job_related['JobDescription_tokenize'].apply(lambda x: [item for item in x if item not in string.punctuation])

data_job_related['RequiredQual_tokenize'] = data_job_related['RequiredQual_tokenize'].apply(lambda x: [item for item in x if item not in stop_words])
data_job_related['RequiredQual_tokenize'] = data_job_related['RequiredQual_tokenize'].apply(lambda x: [item for item in x if item not in string.punctuation])

data_job_related['JobRequirment_tokenize'] = data_job_related['JobRequirment_tokenize'].apply(lambda x: [item for item in x if item not in stop_words])
data_job_related['JobRequirment_tokenize'] = data_job_related['JobRequirment_tokenize'].apply(lambda x: [item for item in x if item not in string.punctuation])

C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [20]:
# lowercase
data_job_related['JobDescription_tokenized'] = data_job_related['JobDescription_tokenize'].apply(lambda x: [item.lower() for item in x])
data_job_related['RequiredQual_tokenized'] = data_job_related['RequiredQual_tokenize'].apply(lambda x: [item.lower() for item in x])
data_job_related['JobRequirment_tokenized'] = data_job_related['JobRequirment_tokenize'].apply(lambda x: [item.lower() for item in x])

C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [21]:
# stem
data_job_related['JobDescription_tokenize'] = data_job_related['JobDescription_tokenize'].apply(lambda x: [PorterStemmer().stem(item) for item in x])
data_job_related['RequiredQual_tokenize'] = data_job_related['RequiredQual_tokenize'].apply(lambda x: [PorterStemmer().stem(item) for item in x])
data_job_related['JobRequirment_tokenize'] = data_job_related['JobRequirment_tokenize'].apply(lambda x: [PorterStemmer().stem(item) for item in x])

C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [22]:
# convert list to string
data_job_related['JobDescription_tokenize_str'] = data_job_related['JobDescription_tokenize'].apply(lambda x: " ".join(x))
data_job_related['RequiredQual_tokenize_str'] = data_job_related['RequiredQual_tokenize'].apply(lambda x: " ".join(x))
data_job_related['JobRequirment_tokenize_str'] = data_job_related['JobRequirment_tokenize'].apply(lambda x: " ".join(x))

C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Akhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [23]:
data_job_related.head()

,Title,JobDescription,RequiredQual,JobRequirment,JobDescription_tokenize,JobRequirment_tokenize,RequiredQual_tokenize,JobDescription_tokenized,RequiredQual_tokenized,JobRequirment_tokenized,JobDescription_tokenize_str,RequiredQual_tokenize_str,JobRequirment_tokenize_str
0,Chief Financial Officer,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\nindividu...",- Supervises financial management and administ...,"[ameria, invest, consult, compani, seek, chief...","[supervis, financi, manag, administr, staff, i...","[To, perform, job, success, individu, must, ab...","[ameria, investment, consulting, company, seek...","[to, perform, job, successfully, individual, m...","[supervises, financial, management, administra...",ameria invest consult compani seek chief finan...,To perform job success individu must abl perfo...,supervis financi manag administr staff includ ...
1,Full-time Community Connections Intern (paid i...,nan,- Bachelor's Degree; Master's is preferred;\n-...,nan,[nan],[nan],"[bachelor, 's, degre, master, 's, prefer, exce...",[nan],"[bachelor, 's, degree, master, 's, preferred, ...",[nan],nan,bachelor 's degre master 's prefer excel skill...,nan
2,Country Coordinator,Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...,"[public, outreach, strengthen, grow, network, ...","[work, countri, director, provid, environment,...","[degre, environment, relat, field, 5, year, re...","[public, outreach, strengthening, growing, net...","[degree, environmentally, related, field, 5, y...","[working, country, director, provide, environm...",public outreach strengthen grow network enviro...,degre environment relat field 5 year relev exp...,work countri director provid environment infor...
3,BCC Specialist,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...,"[the, lead, local, enhanc, develop, health, bc...","[identifi, gap, knowledg, overse, inform, coll...","[advanc, degre, public, health, social, scienc...","[the, lead, local, enhancement, development, h...","[advanced, degree, public, health, social, sci...","[identify, gaps, knowledge, overseeing, inform...",the lead local enhanc develop health bcc speci...,advanc degre public health social scienc commu...,identifi gap knowledg overse inform collect fi...
4,Software Developer,nan,- University degree; economical background is ...,- Rendering technical assistance to Database M...,[nan],"[render, technic, assist, databas, manag, syst...","[univers, degre, econom, background, plu, exce...",[nan],"[university, degree, economical, background, p...","[rendering, technical, assistance, database, m...",nan,univers degre econom background plu excel know...,render technic assist databas manag system rea...


###  Similarity of Jobs based on TF-IDF

In [24]:
tfidf_model = TfidfVectorizer()

In [25]:
Title_tfidf = tfidf_model.fit_transform(data_job_related['Title'])
JobDescription_tfidf = tfidf_model.fit_transform(data_job_related['JobDescription_tokenize_str'])
RequiredQual_tfidf = tfidf_model.fit_transform(data_job_related['RequiredQual_tokenize_str'])
JobRequirment_tfidf = tfidf_model.fit_transform(data_job_related['JobRequirment_tokenize_str'])

In [44]:
def find_similar_jobs_tfidf(index, top_n = 5):
    Title_tfidf_cosine = cosine_similarity(Title_tfidf[index:index+1], Title_tfidf).flatten()
    JobDescription_tfidf_cosine = cosine_similarity(JobDescription_tfidf[index:index+1], JobDescription_tfidf).flatten()
    RequiredQual_tfidf_cosine = cosine_similarity(RequiredQual_tfidf[index:index+1], RequiredQual_tfidf).flatten()
    JobRequirment_tfidf_cosine = cosine_similarity(JobRequirment_tfidf[index:index+1], JobRequirment_tfidf).flatten()

    average_cosine = (Title_tfidf_cosine + JobDescription_tfidf_cosine + RequiredQual_tfidf_cosine + JobRequirment_tfidf_cosine) / 4.0
    #print(average_cosine)
    related_docs_indices = [i for i in average_cosine.argsort()[::-1] if i != index]
    return [(index, average_cosine[index]) for index in related_docs_indices][0:top_n]

In [38]:
cosine_similarity(Title_tfidf[0:1],Title_tfidf).flatten().sum()

485.60260096356427

In [47]:
Title_tfidf  # 18892

<18892x2966 sparse matrix of type '<class 'numpy.float64'>'
	with 60174 stored elements in Compressed Sparse Row format>

In [45]:
data_job_related.iloc[0]

Title                                                    Chief Financial Officer
JobDescription                 AMERIA Investment Consulting Company is seekin...
RequiredQual                   To perform this job successfully, an\nindividu...
JobRequirment                  - Supervises financial management and administ...
JobDescription_tokenize        [ameria, invest, consult, compani, seek, chief...
JobRequirment_tokenize         [supervis, financi, manag, administr, staff, i...
RequiredQual_tokenize          [To, perform, job, success, individu, must, ab...
JobDescription_tokenized       [ameria, investment, consulting, company, seek...
RequiredQual_tokenized         [to, perform, job, successfully, individual, m...
JobRequirment_tokenized        [supervises, financial, management, administra...
JobDescription_tokenize_str    ameria invest consult compani seek chief finan...
RequiredQual_tokenize_str      To perform job success individu must abl perfo...
JobRequirment_tokenize_str  

In [46]:
for index, score in find_similar_jobs_tfidf(0): # average_cosine in the above function is returning the score
    print(data_job_related.iloc[index])         
    print("SIMILARITY SCORE: ", score)
# The Tf-idfVectorizer function is applied on the respective columns without any conditions in them.
# After applying fit_transform of this vectorizer to the columns ,then we are applying cosine similarity on them.
# We are calculating cosine silmilarity for the given index (lets say, 0th index) with the rest all rows or documents in that
# particular column.
# The cosine similarity is giving us a numerical value between the 0th index and rest all other rows and we have filtered the
# the best similarity as top_n = 5.

Title                                                    Chief Financial Officer
JobDescription                 River Island is seeking a Chief Financial Offi...
RequiredQual                   - Master's degree in Management, Finance or Ec...
JobRequirment                  - Develop tools and systems to provide critica...
JobDescription_tokenize        [river, island, seek, chief, financi, offic, r...
JobRequirment_tokenize         [develop, tool, system, provid, critic, financ...
RequiredQual_tokenize          [master, 's, degre, manag, financ, econom, At,...
JobDescription_tokenized       [river, island, seeking, chief, financial, off...
RequiredQual_tokenized         [master, 's, degree, management, finance, econ...
JobRequirment_tokenized        [develop, tools, systems, provide, critical, f...
JobDescription_tokenize_str    river island seek chief financi offic respons ...
RequiredQual_tokenize_str      master 's degre manag financ econom At least 5...
JobRequirment_tokenize_str  

In [34]:
data_job_related.iloc[1]

Title                          Full-time Community Connections Intern (paid i...
JobDescription                                                               nan
RequiredQual                   - Bachelor's Degree; Master's is preferred;\n-...
JobRequirment                                                                nan
JobDescription_tokenize                                                    [nan]
JobRequirment_tokenize                                                     [nan]
RequiredQual_tokenize          [bachelor, 's, degre, master, 's, prefer, exce...
JobDescription_tokenized                                                   [nan]
RequiredQual_tokenized         [bachelor, 's, degree, master, 's, preferred, ...
JobRequirment_tokenized                                                    [nan]
JobDescription_tokenize_str                                                  nan
RequiredQual_tokenize_str      bachelor 's degre master 's prefer excel skill...
JobRequirment_tokenize_str  

In [35]:
for index, score in find_similar_jobs_tfidf(1):
    print(data_job_related.iloc[index])
    print("SIMILARITY SCORE: ", score)

Title                                                    IT Teacher  (full time)
JobDescription                                                               nan
RequiredQual                   - Degree in Computer Science, Information Tech...
JobRequirment                                                                nan
JobDescription_tokenize                                                    [nan]
JobRequirment_tokenize                                                     [nan]
RequiredQual_tokenize          [degre, comput, scienc, inform, technolog, rel...
JobDescription_tokenized                                                   [nan]
RequiredQual_tokenized         [degree, computer, science, information, techn...
JobRequirment_tokenized                                                    [nan]
JobDescription_tokenize_str                                                  nan
RequiredQual_tokenize_str      degre comput scienc inform technolog relat dis...
JobRequirment_tokenize_str  

In [36]:
data_job_related.iloc[2]

Title                                                        Country Coordinator
JobDescription                 Public outreach and strengthening of a growing...
RequiredQual                   - Degree in environmentally related field, or ...
JobRequirment                  - Working with the Country Director to provide...
JobDescription_tokenize        [public, outreach, strengthen, grow, network, ...
JobRequirment_tokenize         [work, countri, director, provid, environment,...
RequiredQual_tokenize          [degre, environment, relat, field, 5, year, re...
JobDescription_tokenized       [public, outreach, strengthening, growing, net...
RequiredQual_tokenized         [degree, environmentally, related, field, 5, y...
JobRequirment_tokenized        [working, country, director, provide, environm...
JobDescription_tokenize_str    public outreach strengthen grow network enviro...
RequiredQual_tokenize_str      degre environment relat field 5 year relev exp...
JobRequirment_tokenize_str  

In [37]:
for index, score in find_similar_jobs_tfidf(2):
    print(data_job_related.iloc[index])
    print("SIMILARITY SCORE: ", score)

Title                                              Country Coordinator - Armenia
JobDescription                 CENN - Caucasus Environmental NGO Network - is...
RequiredQual                   1. Education: University education in environm...
JobRequirment                  -  Write first hand articles, conduct intervie...
JobDescription_tokenize        [cenn, caucasu, environment, ngo, network, loo...
JobRequirment_tokenize         [write, first, hand, articl, conduct, intervie...
RequiredQual_tokenize          [1, educ, univers, educ, environment, scienc, ...
JobDescription_tokenized       [cenn, caucasus, environmental, ngo, network, ...
RequiredQual_tokenized         [1, education, university, education, environm...
JobRequirment_tokenized        [write, first, hand, articles, conduct, interv...
JobDescription_tokenize_str    cenn caucasu environment ngo network look coun...
RequiredQual_tokenize_str      1 educ univers educ environment scienc relat f...
JobRequirment_tokenize_str  